In [1]:
import copy
import os
import argparse
from erasure.utils.logger import GLogger
import torch
import numpy as np
import random
import tracemalloc
from erasure.utils.config.local_ctx import Local
from erasure.utils.config.global_ctx import Global, bcolors 
from erasure.core.factory_base import ConfigurableFactory
from erasure.data.datasets.DatasetManager import DatasetManager
tracemalloc.start()

/NFSHOME/adangelo/miniconda3/envs/representer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
config_file = os.path.join("configs","resource","PROTEINS.jsonc")

In [3]:
global_ctx = Global(config_file)
global_ctx.factory = ConfigurableFactory(global_ctx)

#Create Dataset
data_manager = global_ctx.factory.get_object( Local( global_ctx.config.data ))
global_ctx.dataset = data_manager

2025-05-31 16:28:00,649990589 | INFO | 2636473 - Creating Global Context for: configs/resource/BACE.jsonc
2025-05-31 16:28:00,649990616 | INFO | 2636473 - Setting seeds to: 0
2025-05-31 16:28:00,649990633 | INFO | 2636473 - Caching System: False.
2025-05-31 16:28:00,649990642 | INFO | 2636473 - Instantiating: torch_geometric.datasets.MoleculeNet
2025-05-31 16:28:00,649990724 | INFO | 2636473 - Created Configurable: erasure.data.data_sources.TorchGeometricDataSource.TorchGeometricDataSource
2025-05-31 16:28:00,649990726 | INFO | 2636473 - {'class': 'erasure.data.data_sources.TorchGeometricDataSource.TorchGeometricDataSource', 'parameters': {'datasource': {'class': 'torch_geometric.datasets.MoleculeNet', 'parameters': {'root': 'resources/data', 'name': 'BACE'}, 'preprocess': []}}}
2025-05-31 16:28:01,649991921 | INFO | 2636473 - Instantiating: erasure.data.datasets.DataSplitter.DataSplitterPercentage
2025-05-31 16:28:01,649992009 | INFO | 2636473 - ['all', 'all_shuffled', '-']
2025-05-31

In [4]:
#Create Predictor
current = Local(global_ctx.config.predictor)
current.dataset = data_manager
predictor = global_ctx.factory.get_object(current)
global_ctx.predictor = predictor
global_ctx.logger.info('Global Predictor: ' + str(predictor))

2025-05-31 16:28:03,649994088 | INFO | 2636473 - Instantiating: erasure.model.gcn.DownstreamGCN
2025-05-31 16:28:03,649994098 | INFO | 2636473 - Instantiating: torch.optim.RMSprop
2025-05-31 16:28:03,649994102 | INFO | 2636473 - Instantiating: torch.nn.CrossEntropyLoss
2025-05-31 16:28:03,649994105 | INFO | 2636473 - TRAINING WITH 1210 samples
2025-05-31 16:28:05,649996286 | INFO | 2636473 - epoch = 0 ---> loss = 0.7087	 accuracy = 0.4992
2025-05-31 16:28:07,649998387 | INFO | 2636473 - epoch = 1 ---> loss = 0.6940	 accuracy = 0.5116
2025-05-31 16:28:09,650000516 | INFO | 2636473 - epoch = 2 ---> loss = 0.6896	 accuracy = 0.5281
2025-05-31 16:28:12,650002593 | INFO | 2636473 - epoch = 3 ---> loss = 0.6858	 accuracy = 0.5447
2025-05-31 16:28:14,650004736 | INFO | 2636473 - epoch = 4 ---> loss = 0.6822	 accuracy = 0.5546
2025-05-31 16:28:16,650006818 | INFO | 2636473 - epoch = 5 ---> loss = 0.6792	 accuracy = 0.5679
2025-05-31 16:28:18,650008848 | INFO | 2636473 - epoch = 6 ---> loss = 0

In [ ]:
#Create unlearners 
unlearners = []
unlearners_cfg = global_ctx.config.unlearners
for un in unlearners_cfg:
    current = Local(un)
    current.dataset = data_manager
    current.predictor = copy.deepcopy(predictor)
    unlearners.append( global_ctx.factory.get_object(current) )

#Evaluator
current = Local(global_ctx.config.evaluator)
current.unlearners = unlearners
evaluator = global_ctx.factory.get_object(current)

# Evaluations
for unlearner in unlearners:
    global_ctx.logger.info(f'''{bcolors.OKGREEN}####\t\t Evaluating Unlearner {unlearner.__class__.__name__} \t\t####{bcolors.ENDC}''')
    evaluator.evaluate(unlearner,predictor)

 


2025-05-29 20:27:18,491549508 | INFO | 1627775 - Created Configurable: erasure.unlearners.composite.Identity
2025-05-29 20:27:19,491550204 | INFO | 1627775 - Created Configurable: erasure.unlearners.GoldModel.GoldModel
2025-05-29 20:27:20,491550897 | INFO | 1627775 - Instantiating: torch.optim.Adam
2025-05-29 20:27:20,491550903 | INFO | 1627775 - Created Configurable: erasure.unlearners.Finetuning.Finetuning
2025-05-29 20:27:21,491551628 | INFO | 1627775 - Instantiating: torch.optim.Adam
2025-05-29 20:27:21,491551631 | INFO | 1627775 - Created Configurable: erasure.unlearners.Finetuning.Finetuning
2025-05-29 20:27:21,491552315 | INFO | 1627775 - Instantiating: torch.optim.Adam
2025-05-29 20:27:21,491552318 | INFO | 1627775 - Created Configurable: erasure.unlearners.Finetuning.Finetuning
2025-05-29 20:27:22,491553029 | INFO | 1627775 - Instantiating: torch.optim.Adam
2025-05-29 20:27:22,491553034 | INFO | 1627775 - Created Configurable: erasure.unlearners.SuccessiveRandomLabels.Successi

2025-05-29 20:27:31,491561605 | INFO | 1627775 - Instantiating: torch.optim.Adam
2025-05-29 20:27:31,491561608 | INFO | 1627775 - Created Configurable: erasure.unlearners.NegGrad.NegGrad
2025-05-29 20:27:31,491562289 | INFO | 1627775 - Instantiating: torch.optim.Adam
2025-05-29 20:27:31,491562292 | INFO | 1627775 - Created Configurable: erasure.unlearners.AdvancedNegGrad.AdvancedNegGrad
2025-05-29 20:27:32,491562987 | INFO | 1627775 - Instantiating: torch.optim.Adam
2025-05-29 20:27:32,491562990 | INFO | 1627775 - Created Configurable: erasure.unlearners.AdvancedNegGrad.AdvancedNegGrad
2025-05-29 20:27:33,491563682 | INFO | 1627775 - Instantiating: torch.optim.Adam
2025-05-29 20:27:33,491563685 | INFO | 1627775 - Created Configurable: erasure.unlearners.AdvancedNegGrad.AdvancedNegGrad
2025-05-29 20:27:33,491564359 | INFO | 1627775 - Instantiating: torch.optim.Adam
2025-05-29 20:27:33,491564362 | INFO | 1627775 - Created Configurable: erasure.unlearners.UNSIR.UNSIR
2025-05-29 20:27:33,4

KeyboardInterrupt: 